# Search Engine des alias d'un personnage de Game of Thrones

In [163]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import seaborn as sns
import re
from tqdm import tqdm
sns.set()

SAVE_PATH = "/home/samy/csv_pickle_parquet/" #Le directory ou se trouve les documents d'études


In [164]:
AWOIAF_LIST_URL = "https://awoiaf.westeros.org/index.php/List_of_characters"

HEADERS = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.5993.118 Safari/537.36"}

r = requests.get(AWOIAF_LIST_URL,headers=HEADERS)

soup = BeautifulSoup(r.text)
items = soup.find_all("ul")[9:35]
li_list = []
for ul in items:
    li_list = new_list + ul.find_all("li")


### Création d'un dictionnaire avec tout les liens de téléchargement

In [171]:
characters_url_dict = {}
for item in li_list:
    a = item.find("a")
    title = a.get("title")
    if title:
        href = a.get("href")
        characters_url_dict[title] = "https://awoiaf.westeros.org"+href

dict_keys(['A certain man', 'Abelar Hightower', 'Abelon', 'Addam of Duskendale', 'Addam Frey', 'Addam Hightower', 'Addam Marbrand', 'Addam Osgrey', 'Addam Rivers', 'Addam Velaryon', 'Addam Whitehead', 'Addison Hill', 'Adrack Humble', 'Adrian Redfort', 'Adrian Tarbeck', 'Adrian Thorne', 'Aegon Ambrose', 'Aegon Blackfyre', 'Aegon Frey (son of Stevron)', 'Aegon Frey (son of Aenys)', 'Aegon I Targaryen', 'Aegon II Targaryen', 'Aegon III Targaryen', 'Aegon IV Targaryen', 'Aegon V Targaryen', 'Aegon Targaryen (son of Gaemon)', 'Aegon Targaryen (son of Aenys I)', 'Aegon Targaryen (son of Jaehaerys I)', 'Aegon Targaryen (son of Baelon)', 'Aegon Targaryen (son of Aerys II)', 'Aegon Targaryen (son of Rhaegar)', 'Aegor Rivers', 'Aelinor Penrose', 'Aelor Targaryen', 'Aelora Targaryen', 'Aelyx Targaryen', 'Aemma Arryn', 'Aemon Blackfyre', 'Aemon Costayne', 'Aemon Estermont', 'Aemon Rivers', 'Aemon Steelsong', 'Aemon Targaryen (son of Jaehaerys I)', 'Aemon Targaryen (son of Viserys II)', 'Aemon Targ

### Téléchargement des données et transformation en csv

In [22]:
raw_data_wiki = pd.DataFrame(columns = ["character","raw_data"])

for chara in tqdm(characters_url_dict.keys()):
    fetched_html = requests.get(characters_url_dict[chara],headers=headers).text
    raw_data_wiki.loc[len(raw_data_wiki),:] = [chara,fetched_html]
    
    
raw_data_wiki.to_csv(SAVE_PATH + "awoiaf_raw_html.csv",sep=",",index=0)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3842/3842 [1:07:15<00:00,  1.05s/it]


### Obtention de la page html pour un personnage en particulier

In [166]:
import csv
import codecs
import sys

csv.field_size_limit(sys.maxsize)


FILENAME =SAVE_PATH + "awoiaf_raw_html.csv"
ENCODING = 'utf-8'

#Deux solutions pour lire des données : import via pandas ou streaming

def get_html_streaming(character): #Cette solution est intéressante si la volumétrie de données est très importante (temps d'exec au pire des cas : 3.76s)
    with codecs.open(FILENAME, "r", ENCODING) as fp:
        reader = csv.reader(fp)
        i=0
        for row in tqdm(reader):
            if row[0] == character:
                return row[1]
        raise Exception("Character not found")

                
def get_html_pandas(character): #Solution naive mais la plus efficace dans le cas présent (temps d'exec moyen : 1.72s)
    data = pd.read_csv(FILENAME)
    liste = data[data["character"] == character].raw_data.tolist()
    if liste:
        return liste[0]
    else:
        raise Exception("Character not found")
                

### Obtention des alias d'un personnage

In [167]:
def get_aliases(character):
    aliases_list = []
    html = get_html_streaming(character)
    soup = BeautifulSoup(html)
    aliases_html_th = soup.find("table",class_="infobox").find("th",text="Aliases")
    if aliases_html_th:
        aliases_html_td = aliases_html_th.find_next('td').find_all("li")
        for elmt in aliases_html_td:
            name = elmt.text
            name = ''.join([i for i in name if not i.isdigit()]).replace("[","").replace("]","")
            aliases_list.append(name)
    return aliases_list


###### Cas possibles en résultat du search engine

In [168]:

#1) Personnage inexistant : --> levée d'Exception : personnage non trouvé
get_aliases("Samy") 

3843it [00:03, 982.68it/s] 


Exception: Character not found

In [169]:
#2) Personnage existant mais aucun alias : --> Liste vide
get_aliases("A certain man")

1it [00:00, 430.58it/s]


[]

In [170]:
#3) Personnage existant avec des surnoms : --> Liste de surnoms
get_aliases("Eddard Stark")

1007it [00:01, 873.07it/s]


['Ned', 'The quiet wolf', 'The Ned']